In [105]:
from scipy import linalg 
import numpy as np
import math

In [2]:
tr = np.load('/home/sidd_s/Diffusion_conditional_prior_segmentation/DDPS/tools/saved_files/matrix_numpy/transition_rate_matrix_city.npy')

def betas_tt_arr(betas):
    betas_tt_arr = [np.sum(betas[:(t+1)]) for t in range(betas.shape[0])]
    return np.array(betas_tt_arr)

def simmat(betas_tt, transition_rate):
    matrix = linalg.expm(np.array(betas_tt * transition_rate, dtype=np.float64)) 
    return matrix

### different noise scheduling

In [65]:
# linear scheduler
betas = np.linspace(1e-6, 1e-4,20)
btt = betas_tt_arr(betas)

In [223]:
# btt

In [25]:
## cosine scheduler 

In [26]:
def cosine_fn(step):
      return np.cos((step / num_steps + s) / (1 + s) * np.pi / 2)
def schedule_fn(step):
      return np.clip(1 - (cosine_fn(step + 1) / cosine_fn(step)), 0, 0.999)

In [35]:
s = 1e-9
betas = []
num_steps = 20

In [36]:
betas = [schedule_fn(step) for step  in range(num_steps)]  
betas = np.array(betas)

In [222]:
# betas

In [134]:
def cos_alpha_schedule(time_step, N=20, att_1=0.99999, att_T=0.0001, ctt_1=0.0001, ctt_T=0.99999, exp=3):
    att = np.arange(0, time_step)
    att = (np.cos((att + time_step) * math.pi * 0.5 / time_step) + 1)**exp
    att = att * (att_1 - att_T) + att_T
    att = np.concatenate(([1], att))
    at = att[1:] / att[:-1]

    ctt = np.arange(0, time_step)
    ctt = (np.cos((ctt + time_step) * math.pi * 0.5 / time_step) + 1)**exp
    ctt = ctt * (ctt_1 - ctt_T) + ctt_T
    ctt = np.concatenate(([0], ctt))

    one_minus_ctt = 1 - ctt
    one_minus_ct = one_minus_ctt[1:] / one_minus_ctt[:-1]
    ct = 1 - one_minus_ct
    bt = (1 - at - ct) / N
    att = np.concatenate((att[1:], [1]))
    ctt = np.concatenate((ctt[1:], [0]))
    btt = (1 - att - ctt) / N
    # return at, bt, ct, att, btt, ctt
    return bt 

In [221]:
# betas = cos_alpha_schedule(20)
# betas 

In [225]:
# np.power(np.linspace(1e-4, 1e-1, num=20), math.e)

In [226]:
## testing log space vs linear space

In [229]:
betas = np.logspace(1, 2, base = 10, num=90)
betas

array([ 10.        ,  10.26209321,  10.53105571,  10.80706753,
        11.09031343,  11.38098302,  11.67927086,  11.98537662,
        12.2995052 ,  12.62186688,  12.95267744,  13.29215833,
        13.64053677,  13.99804598,  14.36492526,  14.7414202 ,
        15.12778282,  15.52427173,  15.93115236,  16.34869705,
        16.7771853 ,  17.21690393,  17.6681473 ,  18.13121744,
        18.60642434,  19.09408609,  19.59452913,  20.10808843,
        20.63510778,  21.17593994,  21.73094695,  22.30050032,
        22.8849813 ,  23.48478112,  24.10030129,  24.73195382,
        25.38016154,  26.04535835,  26.72798951,  27.42851197,
        28.14739464,  28.88511875,  29.6421781 ,  30.41907947,
        31.21634289,  32.03450204,  32.87410459,  33.73571256,
        34.61990268,  35.52726683,  36.45841237,  37.41396261,
        38.39455717,  39.40085245,  40.43352204,  41.4932572 ,
        42.5807673 ,  43.69678031,  44.84204325,  46.01732277,
        47.22340555,  48.46109895,  49.73123146,  51.03

In [230]:
betas = np.linspace(10, 100, 90)
betas

array([ 10.        ,  11.01123596,  12.02247191,  13.03370787,
        14.04494382,  15.05617978,  16.06741573,  17.07865169,
        18.08988764,  19.1011236 ,  20.11235955,  21.12359551,
        22.13483146,  23.14606742,  24.15730337,  25.16853933,
        26.17977528,  27.19101124,  28.20224719,  29.21348315,
        30.2247191 ,  31.23595506,  32.24719101,  33.25842697,
        34.26966292,  35.28089888,  36.29213483,  37.30337079,
        38.31460674,  39.3258427 ,  40.33707865,  41.34831461,
        42.35955056,  43.37078652,  44.38202247,  45.39325843,
        46.40449438,  47.41573034,  48.42696629,  49.43820225,
        50.4494382 ,  51.46067416,  52.47191011,  53.48314607,
        54.49438202,  55.50561798,  56.51685393,  57.52808989,
        58.53932584,  59.5505618 ,  60.56179775,  61.57303371,
        62.58426966,  63.59550562,  64.60674157,  65.61797753,
        66.62921348,  67.64044944,  68.65168539,  69.66292135,
        70.6741573 ,  71.68539326,  72.69662921,  73.70

In [231]:
# betas = np.logspace(-13, -9, base = math.e, num=20)
# betas

In [252]:
betas = np.logspace(-5.5, -4.5, base = 10, num=20)
betas

array([3.16227766e-06, 3.56969885e-06, 4.02961132e-06, 4.54877795e-06,
       5.13483291e-06, 5.79639395e-06, 6.54318913e-06, 7.38619982e-06,
       8.33782223e-06, 9.41204967e-06, 1.06246783e-05, 1.19935395e-05,
       1.35387618e-05, 1.52830673e-05, 1.72521055e-05, 1.94748304e-05,
       2.19839265e-05, 2.48162892e-05, 2.80135676e-05, 3.16227766e-05])

In [259]:
# betas[5]

In [220]:
# betas = np.geomspace(1e-6, 1e-4, 20)
# betas 
## similar to np.logspcace 

In [232]:
# betas = np.linspace(3.16e-6, 1e-4,20)
# betas

### testing base matrix with various noise schedulers

In [277]:
m = simmat(betas[6], tr)

In [278]:
m[0]

array([0.05437118, 0.05441255, 0.05425575, 0.05451755, 0.05439195,
       0.05432708, 0.05363509, 0.05476567, 0.05431748, 0.05453235,
       0.05462737, 0.05432987, 0.05413126, 0.05398839, 0.05296079,
       0.04943013, 0.03918479, 0.04378261, 0.05403816])

In [262]:
np.diag(m)

array([0.05263675, 0.05263683, 0.0526356 , 0.05263658, 0.05263602,
       0.0526362 , 0.05264379, 0.05263768, 0.05263596, 0.05263697,
       0.05263654, 0.05263823, 0.05264545, 0.05263633, 0.05265286,
       0.05268067, 0.05445235, 0.05318652, 0.05264033])

In [263]:
m.sum(0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1.])

In [264]:
m.sum(1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1.])

### comparision with confusion matrix

In [21]:
con = np.load('/home/sidd_s/Diffusion_conditional_prior_segmentation/DDPS/tools/saved_files/matrix_numpy/confusion_matrix_cityscapes_segformerb2.npy')

In [22]:
np.diag(con)

array([3.42191258e+08, 4.62292430e+07, 1.94722792e+08, 4.86530400e+06,
       5.28670000e+06, 1.07098340e+07, 1.55824600e+06, 5.43912900e+06,
       1.53996056e+08, 5.48386100e+06, 3.01713920e+07, 1.09683500e+07,
       1.54797300e+06, 5.85725870e+07, 2.46212200e+06, 3.40205200e+06,
       8.91340000e+05, 6.00022000e+05, 5.86972000e+06])

In [25]:
np.diag(tr)

array([ -5628417.,  -7248092., -13844283.,  -2487401.,  -3157585.,
        -4894851.,   -523629.,  -1211845., -11490689.,  -3156210.,
        -1468612.,  -2064963.,   -811242.,  -2678828.,   -401651.,
         -322970.,   -191420.,   -234960.,  -1519438.])

In [23]:
con[0]

array([3.42191258e+08, 2.05861000e+06, 4.92220000e+04, 1.59470000e+04,
       2.46830000e+04, 3.69790000e+04, 1.00000000e+00, 1.24110000e+04,
       4.58740000e+04, 1.01316000e+05, 7.39700000e+03, 1.13438000e+05,
       7.62900000e+03, 4.67588000e+05, 8.48800000e+03, 2.04460000e+04,
       5.30000000e+03, 4.17300000e+03, 5.12500000e+04])

In [24]:
con[2]

array([1.14970000e+04, 2.52640000e+05, 1.94722792e+08, 1.44523000e+05,
       1.85004000e+05, 7.84863000e+05, 9.77150000e+04, 2.29575000e+05,
       3.30994100e+06, 1.56480000e+04, 4.05660000e+05, 2.95075000e+05,
       2.96470000e+04, 1.78111000e+05, 2.47580000e+04, 1.55890000e+04,
       1.21720000e+04, 6.23000000e+03, 1.73833000e+05])